# Animated Maps with country satellite information

Previously, we examined some trends in access to space: how many launches and what payloads were put in orbit lately. Now we'll visualize some of that information in a map. We'll use plotly choropleth Maps. Besides our space launch dataset, we will be using a list of country codes, [available here](https://www.kaggle.com/andradaolteanu/country-mapping-iso-continent-region) with the full name of the country and the continent: in order to properly color a country in the map, we will need the country ISO code. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [ ]:
!pip install chart-studio

import chart_studio.plotly as py
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
init_notebook_mode(connected=True) 
df = pd.read_csv("../input/spacelaunches/space_payloads.csv")
countries = pd.read_csv("../input/country-mapping-iso-continent-region/continents2.csv")

df.head()

As a quick reminder, the "nation" column contains the nation owner of the satellite launched, "country" stores the launch site's country. "first_nation" and "second_nation" are used for satellites that belong to more than one country, or had their ownership transfered.

In [ ]:
countries.head()

# Cleaning

First, time to clean our datasets. For easier comparisons, we'll lowercase the country name. 

In [ ]:
countries["name"] = countries["name"].apply(lambda x : x.lower())

It's necessary to make sure our country names are clean, and join our launch dataset with the country list. In order to do that, we start by looking for countries with unusual characters in their name (any non-letter). 

In [ ]:
df[ df["first_nation"].str.contains("[^a-z ]", regex=True)]

Taking a look at the first_nation and second_nation colums, some contain several countries. Some satellites are jointly opperated. Others may have changed ownership while in orbit.

In [ ]:
df[ df["first_nation"].str.contains("[^a-z ]", regex=True)]["first_nation"].value_counts().tail(30)

To fix this, we use a mask, and split the nation column by any of those characters. Any country after a non-letter character we will send to the "second_country" column. The UK will be handled separately.

In [ ]:
mask_uk =  df["first_nation"].str.contains("u.k.")
df.loc[ mask_uk, "first_nation"] = "uk"
mask =  df["first_nation"].str.contains("[^a-z ]", regex=True)
df.loc[ mask, "second_nation"] = df[mask]["first_nation"].apply(lambda x: re.split("[^a-z ]",x)[1].strip())
df.loc[ mask, "first_nation"] = df[mask]["first_nation"].apply(lambda x: re.split("[^a-z ]",x)[0].strip())

df["first_nation"] = df["first_nation"].str.strip()
df["second_nation"] = df["second_nation"].str.strip()

Now, how many countries are used in the "first_country" column of the dataset, and are absent in the country list? 

In [ ]:
def unmatched_codes(launch_site_codes,site_codes):
    unmatched_code_list = list(set(launch_site_codes)-set(site_codes))
    return unmatched_code_list

In [ ]:
unmatched_codes(df["first_nation"],countries["name"])

Several countries are not in the list. We'll replace the values for USA and the UK, and consider the USSR as Russia. We'll also fix czechoslovakia and North Korea. For satellites that mark their nation as International or Europe, we'll do nothing for the time being. Remember our goal is to visualize the access to space in a map, as the years go by.

After some review of our dataset, the "countries" csv seems to have a mistake: country code "KP" is listed as north korea but should be south korea. We'll fix this as well.

In [ ]:
print(countries[ countries["name"].str.contains("korea") ])
print(countries[ countries["name"].str.contains("czech") ])

df["first_nation"].replace(to_replace={"uk":"united kingdom","usa":"united states","czechoslovakia":"czech republic","ussr":"russia"}, inplace=True)

korea_mask = countries["name"].str.contains("south korea")
skorea_mask = countries["name"].str.contains("korea, republic of")
countries.loc[korea_mask,"name"] = "north korea"
countries.loc[skorea_mask,"name"] = "south korea"
unmatched_codes(df["first_nation"],countries["name"])

Now we are ready to join! We'll create a nations with payloads dataset, joining on the first_nation column. 

Clean the "country" column as well

In [ ]:
 df["country"].value_counts().tail(30)

In [ ]:
df["country"] = df["country"].apply(lambda x: re.split("[/]",x)[0].lower().strip())
df["country"].replace(to_replace={"uk":"united kingdom","usa":"united states","czechoslovakia":"czech republic","ussr":"russia"}, inplace=True)

In [ ]:
df["country"].value_counts().tail(30)

In [ ]:
unmatched_codes(df["country"],countries["name"])

In [ ]:
nation_payloads_with_code = df.join(countries.set_index("name"), on="first_nation", lsuffix="payload_", rsuffix="country_",  how="right")
nation_payloads_with_code.describe()

We seem to have missed some 200 launches (the count for year and region-code is different). Those may be because they are international launches, european, or the country code wasn't registered. Being a bit sloppy, we'll let that slide and go on with our map.

# Plotting

What would we like to see first? First, a few tests to get Choropleth Maps working. Our final goal is an animated map with each passing month, and the space launches that took place at that time.

So we'll begin by preparing our data. We'll be using the period abstraction here to get the information summarized by month. A Period will represent the month and year the launch took place, and it is convenient since we can add and substract months from it easily. Remember the columns we had: alpha-3 has the country cide, year and month the moment of the launch. We are interested in those right now.

We'll group by year, month, and country code, and count how many times a nation has put a payload in space in that period. As "date" we'll keep the first one in the group, since any date will do for calculating the period.

We rename our columns, and create a new year-month column with the period abstraction.

In [ ]:
nation_payloads_with_code.columns

In [ ]:
#Use group by, and create a smaller dataframe with year, month of the launch, the country code, and the number of launches in that period for the selected country.
nations_per_month = nation_payloads_with_code.groupby( ["year","month","alpha-3"]).agg({"nation":"count","date":"first"}).reset_index().rename(columns={"nation":"payloads","alpha-3":"country"})
nations_per_month["year-month"] = pd.to_datetime(nations_per_month["date"]).dt.to_period("M")
nations_per_month["month"] = nations_per_month["month"].astype(int)
nations_per_month["year"] = nations_per_month["year"].astype(int)

#The same, but grouping by year.
nations_per_year = nation_payloads_with_code.groupby( ["year","alpha-3"]).agg({"nation":"count"}).reset_index().rename(columns={"nation":"payloads","alpha-3":"country"})
nations_per_month
#nations_per_year.groupby( ["year"]).agg({"country":"first"}).size().reset_index().rename(columns={0:"payloads","alpha-3":"country"})

These lines are necessary in order to run in Google Colab. In case you are running in Google Colab, this should works. If you are running your jupyter notebook locally, you might need to install the orca package in some other way (just removing the ! sign and running each command individually before each should be enough).

In [ ]:
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install -y xvfb libgtk2.0-0 libgconf-2-4
import sys
!conda install --yes --prefix {sys.prefix} -c plotly plotly-orca 

import plotly.graph_objects as go

Now, in order to display our map we'll use choropleth maps, from plotly. The code should be easy to understand, and we won't cover how to use the library. The documentation [is extensive](https://plotly.com/python/choropleth-maps/) and should help making sense of the code.

We need to build a layout, and our data. In our layout, we'll define the type of projection we want for our map (mercator, in this case), and the title of our plot. In the data, we'll specify how to interpret our dataframe. We'll use only the first month. We need to specify where to get the locations to plot in the map (in this case, country codes), in the location key of the data dictionary. z is the value we want to "paint" that location with. In our case, the number of launches. Colorbar and Colorscale will affect how the color bar is displayed next to the map.

In [ ]:
import chart_studio.plotly as py
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio

init_notebook_mode(connected=False)

first_month = nations_per_month.loc[0].to_frame().transpose()

data = dict(
        type = 'choropleth',
        locations = first_month["country"],
        z = first_month["payloads"],
        text = first_month["payloads"],
         marker = dict(line = dict(color = 'rgb(255,255,255)',width = 1)),
        colorbar = {
            'title' : 'Number of payloads in orbit for the month',
            'x' : 1.2
        },
        colorscale= 'Viridis',
      ) 

layout = dict(
    title = dict(text = 'Satellites put to space in ' + str(first_month.loc[0]["month"]) + '/'+ str(first_month.loc[0]["year"]),
                x = 0.5),
    geo = dict(
        showframe = False,
        projection = {'type':'mercator'}
    ),
    legend = dict(
    xanchor = "left",
    yanchor = "bottom"),
    width = 1000,
    height = 700
    
)

choromap = go.Figure(data = [data],layout = layout)
iplot(choromap, animation_opts={'frame': {'duration': 100}} )


It works! It's not very pretty, but it works. We only displayed the data for the first month in our dataset, in 1957, with the USSR (which we renamed to Russia) launching their first satellite. we'll do a second test now, trying to animate a plot with different frames. There are several ways. We'll be passing several "frames" to the Figure object we build, in this case the 1400th month in our dataframe, and two months after that.

In [ ]:
first_frame = nations_per_month.loc[1400].to_frame().transpose()
second_frame = nations_per_month.loc[1401].to_frame().transpose()
third_frame = nations_per_month.loc[1402].to_frame().transpose()

data = dict(
        type = 'choropleth',
        locations = first_frame["country"],
        z = first_frame["payloads"],
        text = first_frame["payloads"],
         marker = dict(line = dict(color = 'rgb(255,255,255)',width = 1)),
        colorbar = {'title' : 'Total number of payloads'},
                colorscale= 'Portland',

      ) 

data2 = dict(
        type = 'choropleth',
        locations = second_frame["country"],
        z = second_frame["payloads"],
        text = second_frame["payloads"],
         marker = dict(line = dict(color = 'rgb(255,255,255)',width = 1)),
        colorbar = {'title' : 'Total number of payloads'},
                colorscale= 'Portland',

      ) 
data3 = dict(
        type = 'choropleth',
        locations = third_frame["country"],
        z = third_frame["payloads"],
        text = third_frame["payloads"],
         marker = dict(line = dict(color = 'rgb(255,255,255)',width = 1)),
        colorbar = {'title' : 'Total number of payloads'},
                colorscale= 'Portland',

      ) 

layout = dict(
   title = dict(text = 'Number of Satellites per country ',
                x = 0.5),
    geo = dict(
        showframe = False,
        projection = {'type':'mercator'}
    ),
    width = 1000,
    height = 700,
     legend = dict(
    xanchor = "left",
    yanchor = "bottom")
)

#If we wanted an interactive plot, we would use iplot like this. For demonstration purposes, we'll plot each frame separately.
#https://plotly.com/python/getting-started-with-chart-studio/ 
#choromap_multi = go.Figure(data = [data],layout = layout, frames=[
#    go.Frame({'data': data2}),
#    go.Frame({'data':data3}),{'layout':dict(
#    title = 'Second Test',
#    geo = dict(
#        showframe = False,
#        projection = {'type':'mercator'}
#    )
#    , updatemenus=[dict(
#            type="buttons",
#            buttons=[dict(label="Play",
#                          method="animate",
#                          args=[None])])]
#        
#)}])
#iplot(choromap_multi, animation_opts={'frame': {'duration': 100}} )

iplot(go.Figure(data=[data],layout=layout))

In [ ]:
iplot(go.Figure(data=[data2],layout=layout))

In [ ]:
iplot(go.Figure(data=[data3],layout=layout))

The problem with this approach is each frame only shows a single month. It's hard to see a trend if we only visualize what countries launch any given month. It would be better to see thes results accumulate over time, and instead of each of our frames displaying how many satellites were launched at a given months, show instead how much satellites has a given country put to space up to that point.

To build the data we need, first we find all the rows before a given year, group by country, and then sum. For instance:

In [ ]:
rows_1 = nations_per_month[ (nations_per_month["year"]<1960) ].groupby("country").agg({"payloads":"sum","year":"last"})
rows_2 = nations_per_month[ (nations_per_month["year"]<1961) ].groupby("country").agg({"payloads":"sum","year":"last"})
print(rows_1.reset_index())
print(rows_2.reset_index())

But that's not enough, since we also will be using the month. In each of our rows, we have the year and month a payload was launched, as our period. Using that, we can slice our dataset.

In [ ]:
sample_period_dataframe = nations_per_month[ nations_per_month["year-month"]<=pd.Period("1959-1") ]
print(sample_period_dataframe)
print(sample_period_dataframe.groupby("country").agg({"payloads":"sum","year":"last"}))

We will now define a new function putting it all together. With the dataframe and some parameters, we'll build the figure, data, and layout needed to plot. We'll also make the map a bit smaller, hiding Antarctica.

In [ ]:
def setup_map_plot(dataframe, start_year=1957,end_year=2020,  save_images=False, title = "Map"):
    projection = "miller"
    frames = []

    first_year = dataframe[ dataframe["year"]==start_year]

    base_layout = dict(
        title = 'Test',
        geo = dict(
            showframe = False,
            projection = {'type':projection}
        )

    )

    for i in range(start_year,end_year):
        for m in range(1,12):
            row =  dataframe[ dataframe["year-month"]<=pd.Period(str(i)+"-"+str(m)) ].groupby("country").agg({"payloads":"sum","year":"last"}).reset_index()
            
            layout = dict(
                title = dict(text = 'Number of Satellites launched per country in ' + str(m)+'-'+str(i),
                        x = 0.5),
                geo = dict(
                    showframe = False,
                    projection = {'type':projection},
                    lataxis = {"range":[-55,90]}
                ),
            width = 1000,
            height = 700,
             legend = dict(xanchor = "left",yanchor = "bottom")

            )   

            data = dict(
                type = 'choropleth',
                locations = row["country"],
                z = row["payloads"],
                text = row["payloads"],
                 marker = dict(line = dict(color = 'rgb(255,255,255)',width = 1)),
                colorbar = {'title' : title},
                        colorscale= 'Portland',

              ) 
            frames.append(go.Frame({"data":data,"layout":layout}))
            if save_images:
                choro_save = go.Figure(data=[data], layout = layout)
                choro_save.write_image(title+"_"+str(i+m)+".png")
            
    frames.append({'layout':dict(
        title = '',
        geo = dict(
            showframe = False,
            projection = {'type':projection}
        )
        , updatemenus=[dict(
                type="buttons",
                buttons=[dict(label="Play",
                              method="animate",
                              args=[None])])]        
    )})
    
    first_frame = dict(
        type = 'choropleth',
        locations = first_year["country"],
        z = first_year["payloads"],
        text = first_year["payloads"],
         marker = dict(line = dict(color = 'rgb(255,255,255)',width = 1)),
        colorbar = {
            'title' : title,
            'x' : 1.2
        },
                colorscale= 'Viridis',

      ) 

    
    
    return (frames, first_frame, layout)

In [ ]:
frames, data, layout = setup_map_plot(nations_per_month, start_year=1958, end_year=1965, title = "Number of payloads in orbit");

choromap_multi = go.Figure(data=[data], layout = layout, frames=frames)

iplot(choromap_multi, animation_opts={'frame': {'duration': 100}} )
#pio.write_html(choromap, file='hello_world_2.html', auto_open=True)

This is not too bad, but since Russia and the US account for a significant number of the launches, we can not really see any new trends emerging, for example in the last few years. If we take a look at the top 5 countries, we notice the imbalance.

In [ ]:
nations_per_month.groupby("country").agg({"payloads":"sum"}).sort_values("payloads",ascending=False).head(5).plot(kind="bar")

We'll try solving this using a running average of the payloads in the last few years, and plot that. We'll use a "sliding window" of valid periods, just using pd.Period + N, with N the number of months we'd like to consider.

In [ ]:
starting_period = pd.Period("1970-1")
months = 12

print("12 month period since 1970")
print(nations_per_month[  ( nations_per_month["year-month"]>=starting_period ) & ( nations_per_month["year-month"]<=starting_period + months ) ].groupby("country").agg({"payloads":"sum","year":"last"}).reset_index())

#Now, we'll run this for 2 years, one month at a time, and get 24 different dataframes.

test_df = pd.DataFrame()

for i in range(12):
    period = starting_period + i
    tdf = nations_per_month[  ( nations_per_month["year-month"]>=period ) & ( nations_per_month["year-month"]<=period + months ) ].groupby("country").agg({"payloads":"sum","year":"last"}).reset_index()
    tdf["frame"] = i;
    test_df = test_df.append(tdf)

fig = plt.figure(figsize=(14,10))

fig = sns.lineplot(data=test_df,y="payloads",x="frame", hue="country")

There's still a big difference, but we'll modify our function and see how that works out. We'll be passing the portion of code that creates the frame as a callback, in order to test several window sizes. And, as you might have noticed, the scale varies wildly from one moment to the other. That's not useful, so we'll fix the minimum and maximum values of z, checking the global max when we compute our data.

Furthermore, in order to deal with the large difference in values from our scale, we won't use a continuous color map, but segment our scale conviniently to have the smallest values use a different color.

To do this, we compute the smallest value in our set of data, and map that value to a 0-1 scale. (the input for the color scale is an array [0 to 1, [rgb] ]. So any value below 0.9 (we'll see why later on) will be the lowest "threshold" in our scale, and we'll paint that gray. From there, we'll use a green -> blue color scale. It goes from [0,255,0] to [0,255,255], and then removes green to have only blue in the higher values: [0,0,255]

That scale won't be linear, however: the lowest 20% will account for most of our color values. Once a country is above a certain threshold, we don't really care about the difference The upper 80% we'll then have most of the higher, "only blue" values.

In [ ]:
def setup_map_plot(dataframe, frame_calculation, start_year=1957,end_year=2020, chart_title = "Title", title = "Map", filename="map", save_images=False, **kwargs):
    projection = "miller"
    frames = []

    first_year = dataframe[ dataframe["year"]==start_year]

    base_layout = dict(
        title = 'Test',
        geo = dict(
            showframe = False,
            projection = {'type':projection}
        )

    )
    
    z_min = 0;
    z_max = 0;

    rows = [frame_calculation(dataframe,i,m,extra_args=kwargs) for i in range(start_year,end_year) for m in range(1,12)]
    
    for i in range(start_year,end_year):
        for m in range(1,12):
            row =  frame_calculation(dataframe,i,m,extra_args=kwargs)
            z_max = max(z_max,row["payloads"].max())
    
        
    data = dict(
        type = 'choropleth',
        locations = first_year["country"],
        z = first_year["payloads"],
        zmax = z_max,
        text = first_year["payloads"],
         marker = dict(line = dict(color = 'rgb(255,255,255)',width = 1)),
        colorbar = {'title' : title},
                colorscale= 'Wistia',

      ) 
    layout = dict(
        title = dict(text = chart_title,
                x = 0.5),
        geo = dict(
            showframe = False,
            projection = {'type':projection},
            lataxis = {"range":[-55,90]}
        ),
    width = 1000,
    height = 700,
     legend = dict(xanchor = "left",yanchor = "bottom")

    )   
    
    z_range = (z_max-z_min)

    for i,row in enumerate(rows):
        data = dict(
                type = 'choropleth',
                locations = row["country"],
                z = row["payloads"],
                zmax = z_max,
                zmin = z_min,
                text = row["payloads"],
                 marker = dict(line = dict(color = 'rgb(255,255,255)',width = 1)),
                colorbar = {'title' : title,'x':1.2},
                colorscale = "portland",

              )
        
        layout["title"]["text"] = chart_title + str((1+i)%12)+'-'+str( (start_year)+(i//12) )
        frames.append(go.Frame({"data":data,"layout":layout}))
        
        if save_images:
            choro_save = go.Figure(data=[data], layout = layout)
            choro_save.write_image(str(i)+"_gif_"+filename+".png")

    frames.append({'layout':dict(
        title = title,
        geo = dict(
            showframe = False,
            projection = {'type':projection}
        )
        , updatemenus=[dict(
                type="buttons",
                buttons=[dict(label="Play",
                              method="animate",
                              args=[None])])]        
    )})
  

    return (frames, data, layout)

These are the functions that will create the rows to plot in our map. To make it more usable later, we'll allow the target column (the one we'll sum) as a parameter

In [ ]:
def cummulative_rows(dataframe,year,month, extra_args):
    rows =  dataframe[ dataframe["year-month"]<=pd.Period(str(year)+"-"+str(month)) ].groupby("country").agg({"payloads":"sum","year":"last"}).reset_index()
    return rows;

def running_average_rows(dataframe,year,month, extra_args):
    months = extra_args["months"]
    starting_period = pd.Period(str(year)+"-"+str(month))
    period = starting_period + months
    rows = dataframe[  ( dataframe["year-month"]>starting_period ) & ( dataframe["year-month"]<=period ) ].groupby("country").agg({"payloads":"sum","year":"last"}).reset_index()
    return rows;




In [ ]:
window_size = 48

frames, data, layout = setup_map_plot(nations_per_month, running_average_rows, start_year = 2012, end_year = 2018,
                                      chart_title = 'Satellites belonging to each country launched in the ' + str(window_size) + " months after ",
                                      title="Satellites put in orbit last "+ str(window_size) + " months", filename="running_avg_4_years", save_images=False, months=window_size, column="payload");
choromap_multi = go.Figure(data=[data], layout = layout, frames=frames)



iplot(choromap_multi, animation_opts={'frame': {'duration': 500}} )

That's pretty good. It would also be nice if our map could "remember" what countries have launched in the past, to be able to tell them apart from countries that haven't launched yet. In order to do that, we'll define a new function and pass it as a parameter for our map plot function: It'll return the running average, but countries that have launched in the past will be included with "0.1" payloads.

In [ ]:
months = 12
year = 2012
month = 4
starting_period = pd.Period(str(year)+"-"+str(month))
period = starting_period + months
rows_window = nations_per_month[  ( nations_per_month["year-month"]>starting_period ) & ( nations_per_month["year-month"]<=period ) ].groupby("country").agg({"payloads":"sum","year":"last"}).reset_index()
rows_cummulative = nations_per_month[  ( nations_per_month["year-month"]<=starting_period ) ].groupby("country").agg({"payloads":"sum","year":"last"}).reset_index()
rows_cummulative["payloads"] = 0.01
rows_window["payloads"] = rows_window["payloads"]*10
merged = rows_cummulative.merge(rows_window,how="outer",on="country",suffixes=("_window","_cummulative"))
merged = merged.fillna(0)
merged.head(10)

To have our slice of dataframe ready, we'll keep the larger value between the cummulative and the window values, for the "payloads" column.

In [ ]:
merged["year"] = merged["year_window"].astype(int)
merged["payloads"] = merged[ ["payloads_cummulative","payloads_window"] ].max(axis=1)
merged = merged[ ["year","payloads","country"] ]
merged.head()

Putting it all together:

In [ ]:
def running_average_with_memory_rows(dataframe,year,month, extra_args):
    months = extra_args["months"]
    starting_period = pd.Period(str(year)+"-"+str(month))
    period = starting_period + months
    rows_window = dataframe[  ( dataframe["year-month"]>starting_period ) & ( dataframe["year-month"]<=period ) ].groupby("country").agg({"payloads":"sum","year":"last"}).reset_index()
    rows_cummulative = dataframe[  ( dataframe["year-month"]<=starting_period ) ].groupby("country").agg({"payloads":"sum","year":"last"}).reset_index()
    rows_cummulative["payloads"] = 0.01
    merged = rows_cummulative.merge(rows_window,how="outer",on="country",suffixes=("_window","_cummulative"))
    merged = merged.fillna(0)
    merged["year"] = merged["year_window"].astype(int)
    merged["payloads"] = merged[ ["payloads_cummulative","payloads_window"] ].max(axis=1)
    return merged[ ["year","payloads","country"] ]
    


In [ ]:
dec_2019_satellites = running_average_with_memory_rows(nations_per_month,2019,12,extra_args={'months': 12})
dec_2019_satellites.tail(10)

One extra thing: redifining the function to plot the log of the payloads: The vast majority of our values are between 0 and 20, with few above 100 and 400. With a logarithmic scale we'll be able to visualize that information better. We'll define a function to transform our payload, keeping our fake 0.01 value to signify past launches as 0, a single launch as log(1.1), and log of the number of payloads for the rest.

In [ ]:
dec_2019_satellites["payloads"].hist(bins=30)

In [ ]:
def transform_payload_count(count):
    if count == 0.01:
        return np.log10(1)
    elif count == 1:
        return np.log10(1.1)
    else:
        return np.log10(count)

In [ ]:
dec_2019_satellites["payloads"].apply(transform_payload_count)

In [ ]:
def setup_map_plot(dataframe, frame_calculation, start_year=1957,end_year=2020, chart_title = "Title", title = "Map", filename="map", save_images=False, **kwargs):
    projection = "miller"
    frames = []

    first_year = dataframe[ dataframe["year"]==start_year]

    base_layout = dict(
        title = 'Test',
        geo = dict(
            showframe = False,
            projection = {'type':projection}
        )

    )
    
    z_min = 0;
    z_max = 0;

    rows = [frame_calculation(dataframe,i,m,extra_args=kwargs) for i in range(start_year,end_year) for m in range(1,12)]
    
    for i in range(start_year,end_year):
        for m in range(1,12):
            row =  frame_calculation(dataframe,i,m,extra_args=kwargs)
            z_max = max(z_max,row["payloads"].apply(transform_payload_count).max())

    layout = dict(
        title = dict(text = chart_title,
                x = 0.5),
        geo = dict(
            showframe = False,
            projection = {'type':projection},
            lataxis = {"range":[-55,90]}
        ),
    width = 800,
    height = 600,
     legend = dict(xanchor = "left",yanchor = "bottom")

    )   
    tick_values = [0,0.2,0.4,0.6,1]
    z_range = (z_max-z_min)
    #smallest_value = (0.9)/(z_range)
    smallest_value = 0.001
    print("factor is ", smallest_value)
    for i,row in enumerate(rows):
        data = dict(
                type = 'choropleth',
                locations = row["country"],
                z = row["payloads"].apply(transform_payload_count),
                zmax = z_max,
                zmin = z_min,
                text = row["payloads"],
                 marker = dict(line = dict(color = 'rgb(255,255,255)',width = 1)),
                colorbar = {'title' : title,'x':1.2,'tickmode': "array",
                            'tickvals': [tval * z_range for tval in tick_values],
                            'ticktext': [np.power(10,tval * z_range).astype(int) for tval in tick_values]
                           },
                colorscale = [[0, 'rgb(90,90,90, 0.5)'],
                      [smallest_value, 'rgb(90,90,90,0.5)'],
                      [smallest_value, 'rgb(0,255,0,0.5)'],
                      [0.5, 'rgb(0,255,255,0.5)'],
                      [1, 'rgb(0,0,255,0.5)'],
                     ],

              )
        
        layout["title"]["text"] = chart_title + str(1+(i)%12)+'-'+str( (start_year)+(i//12) )
        frames.append(go.Frame({"data":data,"layout":layout}))
        
        if save_images:
            choro_save = go.Figure(data=[data], layout = layout)
            choro_save.write_image(str(i)+"_gif_"+filename+".png")

    frames.append({'layout':dict(
        title = title,
        geo = dict(
            showframe = False,
            projection = {'type':projection}
        )
        , updatemenus=[dict(
                type="buttons",
                buttons=[dict(label="Play",
                              method="animate",
                              args=[None])])]        
    )})
  

    return (frames, data, layout)

In [ ]:
window_size = 24

frames, data, layout = setup_map_plot(nations_per_month, running_average_with_memory_rows, start_year = 1957, end_year = 2020,
                                      chart_title = 'Satellites belonging to each country launched in the ' + str(window_size) + " months after ",
                                      title="Satellites in orbit last "+ str(window_size) + " months",filename="running_avg_memory_2_years",save_images=False, months=window_size );
choromap_multi = go.Figure(data=[data], layout = layout, frames=frames)



iplot(choromap_multi, animation_opts={'frame': {'duration': 100}} )

Finally, we'll create a gif from the output images, using our "save_images" parameters we'll create a series of png files for each frame of our map, and combine for export. Some of the files can be a bit large!

In [ ]:
!pip install Pillow

In [ ]:
setup_map_plot(nations_per_month, running_average_with_memory_rows, start_year = 1957, end_year = 2020,
                                      chart_title = 'Satellites belonging to each country launched in the ' + str(window_size) + " months after ",
                                      title="Satellites in orbit last "+ str(window_size) + " months",filename="1_running_avg_memory_2_years",save_images=True, months=24 );

setup_map_plot(nations_per_month, running_average_rows, start_year = 1957, end_year = 2020,
                                      chart_title = 'Satellites belonging to each country launched in the ' + str(window_size) + " months after ",
                                      title="Satellites put in orbit last "+ str(window_size) + " months", filename="2_running_avg_2_years", save_images=True, months=24);


In [ ]:
!ls *2_running_avg_2_years*.png

In [ ]:
import glob
from PIL import Image

gif_1 = "1_running_avg_memory_2_years"
gif_2 = "2_running_avg_2_years"

'''Build a gif with all the images starting with the given prefix.
'''
def pngs_to_gif(file_prefix):
    # filepaths
    fp_in = "./*_gif_"+file_prefix+"*.png"
    fp_out = "gif_"+file_prefix+"_image.gif"

    # https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
    img, *imgs = [Image.open(f) for f in ["./"+f.split("_gif_")[0].replace("./","")+"_gif_"+file_prefix+".png" for f in sorted([f.split("_gif_")[0].replace("./","") for f in glob.glob(fp_in)], key=int) ] ]
    img.save(fp=fp_out, format='GIF', append_images=imgs,
             save_all=True, duration=200, loop=0)
    
pngs_to_gif(gif_1)
pngs_to_gif(gif_2)

In [ ]:
!ls -ltr *.gif

Finally, we build a link to be able to download the files

In [ ]:

from IPython.display import FileLink
FileLink(r'gif_'+gif_1+'_image.gif')


In [ ]:
FileLink(r'gif_'+gif_2+'_image.gif')

In [ ]:
!rm -rf *.png